In [1]:
import pymysql 
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
from tqdm import tqdm
from collections import defaultdict, Counter
import _pickle as pickle
import random
from scipy.stats import epps_singleton_2samp, wasserstein_distance, ks_2samp
import matplotlib.pyplot as plt
import seaborn as sns
from random import sample

In [6]:
# start tunnel first: ssh -f [uni]@mimir.dbmi.columbia.edu -L 3307:127.0.0.1:3306 -N

conn = pymysql.connect(host="127.0.0.1", 
                       user="", #uni
                       port = ,
                       passwd="", #sql password
                       db = "" ) #database
cur = conn.cursor()

In [3]:
cur.execute('''select phecode, concept_id_2
                from clinical_merge_v5_2022q1.phecode_icd10
                join concept on icd10 = concept_code
                join concept_relationship on concept_id = concept_id_1 
                where vocabulary_id = 'ICD10CM'
                and phecode != '';''')

phe_condition = pd.DataFrame([i for i in cur.fetchall()], columns=['phecode', 'concept_id'])

In [4]:
len(phe_condition)

131652

In [7]:
cur.execute('''select pat_mrn_id, condition_concept_id, condition_start_date
                from condition_occurrence
                inner join covid_patient2person on person_id_2020 = person_id;''')

prev_condition_data = pd.DataFrame([i for i in cur.fetchall()], columns=['pat_mrn_id', 'concept_id', 'condition_start'])

In [9]:
len(prev_condition_data)

65472496

In [21]:
prev_condition_data_filtered = prev_condition_data[prev_condition_data['concept_id'].isin(list(phe_condition['concept_id']))]
len(prev_condition_data_filtered)

41234815

In [22]:
prev_phe_data = prev_condition_data_filtered.merge(phe_condition, how='inner', left_on = 'concept_id', right_on='concept_id')[['pat_mrn_id', 'phecode', 'condition_start']]


In [23]:
len(prev_phe_data)

182783794

In [24]:
prev_phe_data.to_csv('data/clinical_merge_v5_2020q4_condition_occurrence.csv', index=False)

In [25]:
cur.execute('''select *
                from user_vr2430.vfinal_1_predict_covid_visits;''')

visit_data = pd.DataFrame([i for i in cur.fetchall()], columns=['visit_id','pat_mrn_id', 'st_dt', 'ed_dt'])


In [30]:
prev_phe_data_visit = prev_phe_data.merge(visit_data, how='inner', left_on = 'pat_mrn_id', right_on = 'pat_mrn_id')

In [33]:
len(prev_phe_data_visit)

817275291

### kernel died as I was exporting table. Just repeating the last step and writing to csv file

In [1]:
import pymysql 
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
from tqdm import tqdm
from collections import defaultdict, Counter
import _pickle as pickle
import random
from scipy.stats import epps_singleton_2samp, wasserstein_distance, ks_2samp
import matplotlib.pyplot as plt
import seaborn as sns
from random import sample

In [2]:
# start tunnel first: ssh -f [uni]@mimir.dbmi.columbia.edu -L 3307:127.0.0.1:3306 -N

conn = pymysql.connect(host="127.0.0.1", 
                       user="", #uni
                       port = ,
                       passwd="", #sql password
                       db = "" ) #database
cur = conn.cursor()

In [3]:
cur.execute('''select *
                from user_vr2430.vfinal_1_predict_covid_visits;''')

visit_data = pd.DataFrame([i for i in cur.fetchall()], columns=['visit_id','pat_mrn_id', 'st_dt', 'ed_dt'])


In [12]:
prev_phe_data = []
file = open('data/clinical_merge_v5_2020q4_condition_occurrence.csv', 'r')

for line in tqdm(file):
    if 'pat_mrn_id' in line:
        continue
    mrn = int(line.split(',')[0])
    phe = float(line.split(',')[1])
    dt = datetime.strptime(line.strip().split(',')[2], '%Y-%m-%d').date()
    prev_phe_data.append([mrn, phe, dt])
    
file.close()
len(prev_phe_data)

182783795it [27:58, 108891.59it/s]


182783794

In [13]:
prev_phe_data = pd.DataFrame(prev_phe_data, columns=['pat_mrn_id','phecode','condition_start'])

In [14]:
prev_phe_data_visit = prev_phe_data.merge(visit_data, how='inner', left_on = 'pat_mrn_id', right_on = 'pat_mrn_id')

In [15]:
prev_phe_data_visit_filtered = prev_phe_data_visit[prev_phe_data_visit.condition_start < prev_phe_data_visit.st_dt]

In [20]:
prev_phe_data_visit_filtered = prev_phe_data_visit_filtered[['phecode', 'visit_id', 'condition_start']]

In [22]:
prev_phe_data_visit_filtered.to_csv('data/previous_conditions_phe.csv', index=False)